# 📘 代理架构 16：元胞自动机

欢迎来到我们系列的第十六个notebook。今天，我们探索**元胞自动机**架构，这是一种受复杂系统理论启发的模式。

元胞自动机展示了简单规则如何产生复杂的涌现行为，这对于理解和设计分布式代理系统很有价值。

### 定义
一个**Grid-Based Agent System** 是一个架构 其中大量的简单的 代理(or "cells") 被排列在一个spatial grid. 每个代理 有一个state和updates它synchronously based在一个rule set 那个only considers states其immediate neighbors. Complex, high-level 模式s和problem-solving capabilities emerge从这些local interactions.

### 高级工作流程

1. **Grid Initialization:** 一个grid的cell-agents 是created, each 已初始化与一个type (e.g., obstacle, empty)和一个state (e.g., 一个value).
2. **Set Boundary Conditions:** One或more cells 是given 一个special state来start 一个computation (e.g., 一个"target" cell's value 是set来0).
3. **Synchronous Tick:** 系统 "ticks" forward. 在each tick, every cell simultaneously 计算它的next状态基于 current state其neighbors.
4. **Emergence:** 作为系统 ticks,信息propagates across grid like 一个wave. 这可以创建gradients, paths,和other 复杂的 structures.
5. **State Stabilization:** 系统 runs until grid状态stabilizes (no more changes occur), indicating 那个computation 是完成.
6. **Readout:** 该解决方案来problem 是then read directly从final state的grid (e.g.,通过following 一个computed gradient).

### 何时使用/应用场景
* **Spatial Reasoning & Logistics:** Optimal pathfinding在dynamic environments (like我们的warehouse example).
* **Complex System Simulation:** Modeling phenomena与emergent behavior like forest fires, disease spread,或urban growth.
* **Parallel Computation:** Certain algorithms 可以be mapped一个cellular automata 模型为execution在highly parallel hardware (like GPUs).

### 优点和缺点
* **优点：**
 * **High Parallelism:** logic 是inherently parallel, making它extremely fast在ppropriate hardware.
 * **Adaptability:** 系统 可以dynamically react来changes在environment (e.g., 一个new obstacle)通过simply re-propagating它的waves.
 * **Emergent Complexity:** Can解决very 复杂的 problems与surprisingly 简单的 rules.
* **缺点：**
 * **Design Complexity:** Designing local rules来produce desired global behavior 可以be challenging和unintuitive.
 * **Less Introspective:** It's harder来ask 一个single cell "why"它有一个certain state; reasoning 是distributed across entire 系统.

## 阶段0：基础与设置

我们need `numpy`为高效的 grid operations和`rich`为high-quality terminal visualizations.

In [1]:
# !pip install -q -U langchain-nebius rich python-dotenv numpy

In [ ]:
import os
import numpy as np
import time from typing import List, Dict, Any, Optional, Tuple from dotenv import load_dotenv fromIPython.display import clear_output

# LangChainforoptional final summary from langchain_nebius import ChatNebius from langchain_core.prompts import ChatPromptTemplate

# 用于美观打印andvisualization from rich.console import Console from rich.table import Table from rich.panel import Panel

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Cellular Automata (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：构建元胞自动机环境

这是最关键的阶段. 我们将定义 two core classes用于我们的simulation:
1. `CellAgent`: Represents 一个single cell在grid. It contains它的type, its状态(a pathfinding value),和local rule为updating 那个状态。
2. `WarehouseGrid`: container为entire 系统. It will管理2D array的`CellAgent`s, orchestrate synchronous `tick` updates,和handle visualization.

In [3]:
console = Console()

class CellAgent:
 """A single 代理inour grid. Its only job is来update its value basedinneighbors."""
 def __init__(self, cell_type: str, item: Optional[str] = None):
 self.type = cell_type # 'EMPTY', 'OBSTACLE', 'SHELF', 'PACKING_STATION'
 self.item = item
 self.pathfinding_value = float('inf')

 def update_value(self, neighbors: List['CellAgent']):
 """core local rule: my新的value is 1 + minimum value的my non-obstacle neighbors."""
 if self.type == 'OBSTACLE':
 returnfloat('inf')
 
 min_neighbor_value = float('inf')
 forneighborinneighbors:
 if neighbor.pathfinding_value < min_neighbor_value:
 min_neighbor_value = neighbor.pathfinding_value
 
 # +1 represents cost的movingfrom一个neighbor来这个cell
 returnmin(self.pathfinding_value, min_neighbor_value + 1)

class WarehouseGrid:
 """Manages entire grid的CellAgentsandsimulation loop."""
 def __init__(self, layout: List[str]):
 self.height = len(layout)
 self.width = len(layout[0])
 self.grid = self._create_grid_from_layout(layout)
 self.item_locations = self._get_item_locations()

 def _create_grid_from_layout(self, layout):
 grid = np.empty((self.height, self.width), dtype=object)
 forr, row_strinenumerate(layout):
 forc, charinenumerate(row_str):
 if char == ' ':
 grid[r, c] = CellAgent('EMPTY')
 elif char == '#':
 grid[r, c] = CellAgent('OBSTACLE')
 elif char == 'P':
 grid[r, c] = CellAgent('PACKING_STATION')
 else: # It's 一个item
 grid[r, c] = CellAgent('SHELF', item=char)
 returngrid

 def _get_item_locations(self) -> Dict[str, Tuple[int, int]]:
 locations = {}
 forrinrange(self.height):
 forcinrange(self.width):
 if self.grid[r, c].type == 'SHELF':
 locations[self.grid[r, c].item] = (r, c)
 if self.grid[r, c].type == 'PACKING_STATION':
 locations['P'] = (r, c)
 returnlocations

 def get_neighbors(self, r: int, c: int) -> List[CellAgent]:
 neighbors = []
 fordr, dcin[(0, 1), (0, -1), (1, 0), (-1, 0)]: # N, S, E, W
 nr, nc = r + dr, c + dc
 if 0 <= nr < self.heightand0 <= nc < self.width:
 neighbors.append(self.grid[nr, nc])
 returnneighbors

 def tick(self) -> bool:
 """执行 one synchronous update的all cells. return True if any value changed."""
 changed = False
 # First, calculate all新的values 基于 current state
 new_values = np.empty((self.height, self.width))
 forrinrange(self.height):
 forcinrange(self.width):
 neighbors = self.get_neighbors(r, c)
 new_values[r, c] = self.grid[r, c].update_value(neighbors)
 
 # Then,应用all新的values
 forrinrange(self.height):
 forcinrange(self.width):
 if self.grid[r, c].pathfinding_value != new_values[r, c]:
 self.grid[r, c].pathfinding_value = new_values[r, c]
 changed = True
 returnchanged

 def visualize(self, show_values: bool = False, title: str = "Warehouse Grid"):
 """Displays gridstateusing Rich."""
 table = Table(title=title, show_header=False, show_edge=True, padding=0)
 for_inrange(self.width):
 table.add_column(justify="center")
 
 forrinrange(self.height):
 row_renderables = []
 forcinrange(self.width):
 cell = self.grid[r, c]
 val = cell.pathfinding_value
 display_char = ''
 if cell.type == 'EMPTY': display_char = '[grey70]·[/grey70]'
 elif cell.type == 'OBSTACLE': display_char = '[red]█[/red]'
 elif cell.type == 'PACKING_STATION': display_char = '[bold green]P[/bold green]'
 elif cell.type == 'SHELF': display_char = f'[bold blue]{cell.item}[/bold blue]'

 if show_valuesandval != float('inf'):
 # Color code path values
 color = int(255 - (val * 5) % 255)
 row_renderables.append(f"[rgb({color},{color},{color})inrgb(30,30,60)]{int(val):^3}[/]")
 else:
 row_renderables.append(f" {display_char} ")
 table.add_row(*row_renderables)
 console.print(table)

print("细胞自动机环境defined successfully。")

Cellular Automata environment 定义成功.


## 阶段2： 实现 Emergent Behaviors

grid itself 是just 一个framework. 我们need来实现high-level logic 那个uses cellular automata来solve我们的problem. 这包括 two key emergent behaviors:

1. **Path Wave Propagation:** A函数那个sets 一个target和lets grid `tick` until 一个完成 pathfinding gradient 有formed across entire warehouse floor.
2. **Gradient Descent Traversal:** A函数那个simulates 一个"carrier" 代理 starting在一个item's shelf和simply following path的steepest descent (lowest `pathfinding_value`) 直到它 reaches target.

In [4]:
def propagate_path_wave(grid: WarehouseGrid, target_pos: Tuple[int, int], visualize_steps: bool = False):
 """Resetsandthen runs simulation until pathfinding values stabilize."""
 # Reset all pathfinding values
 forrinrange(grid.height):
 forcinrange(grid.width):
 grid.grid[r, c].pathfinding_value = float('inf')
 
 # Set target's value来0来start wave
 grid.grid[target_pos].pathfinding_value = 0
 
 tick_count = 0
 while True:
 tick_count += 1
 if visualize_steps:
 clear_output(wait=True)
 grid.visualize(show_values=True, title=f"Path Wave Propagation (Tick #{tick_count})")
 time.sleep(0.1)
 
 changed = grid.tick()
 if not changed:
 break
 if visualize_steps:
 clear_output(wait=True)
 grid.visualize(show_values=True, title=f"Path Wave Propagation (StabilizedinTick #{tick_count})")

def trace_and_move_item(grid: WarehouseGrid, start_pos: Tuple[int, int]) -> List[Tuple[int, int]]:
 """Follows gradientfromstart position back来target (value 0)."""
 path = [start_pos]
 r, c = start_pos
 
 while grid.grid[r, c].pathfinding_value > 0:
 neighbors = grid.get_neighbors(r, c)
 best_neighbor_pos = None
 min_val = grid.grid[r, c].pathfinding_value
 
 # Find neighborwithlowest pathfinding value
 forneighbor_cellinneighbors:
 # Find position的neighbor cell
 pos_list = np.where(grid.grid == neighbor_cell)
 if len(pos_list[0]) > 0:
 nr, nc = pos_list[0][0], pos_list[1][0]
 if neighbor_cell.pathfinding_value < min_val:
 min_val = neighbor_cell.pathfinding_value
 best_neighbor_pos = (nr, nc)
 
 if best_neighbor_pos:
 path.append(best_neighbor_pos)
 r, c = best_neighbor_pos
 else:
 console.print("[red]error： 路径追踪卡住了。未找到下坡邻居。[/red]")
 break
 
 returnpath

print("涌现行for函数defined successfully。")

Emergent behavior函数定义成功.


## 阶段3： Full Orchestration 工作流程

Now 我们将创建top-level函数那个simulates entire order fulfillment 过程的核心。这will 演示how emergent behaviors 可以be composed来solve 一个multi-step problem.

In [5]:
def fulfill_order(layout: List[str], order: List[str], visualize_waves: bool = False):
 """main orchestration function."""
 grid = WarehouseGrid(layout)
 console.print("--- 初始仓库state ---")
 grid.visualize()
 
 packing_station_pos = grid.item_locations['P']
 
 fori, item_idinenumerate(order):
 panel_title = f"[bold]步骤 {i+1}: Fulfill Item '{item_id}'[/bold]"
 log_messages = []
 
 item_pos = grid.item_locations.get(item_id)
 if not item_pos:
 console.print(Panel(f"[red]error：仓库中未找到物品'{item_id}'。[/red]", title=panel_title))
 continue
 
 # 1. Compute path wavefrompacking station
 log_messages.append("🌊 Computing path wavefromPacking Station...")
 propagate_path_wave(grid, packing_station_pos, visualize_steps=visualize_waves)
 
 # 2. Trace pathforcurrent item
 log_messages.append(f"🚚 Found pathforitem {item_id}. Moving along gradient...")
 path = trace_and_move_item(grid, item_pos)
 path_str = ' -> '.join(map(str, path))
 log_messages.append(f"Path: {path_str}")

 # 3. Update gridstate(item is nowinpacking station)
 grid.grid[item_pos].type = 'EMPTY'
 grid.grid[item_pos].item = None
 log_messages.append(f"✅ Item '{item_id}' has been moved来packing station.")
 console.print(Panel('\n'.join(log_messages), title=panel_title, border_style="blue"))
 
 console.print(Panel(f"system successfully fulfilled orderforitems {order}passemergently computing paths through local cell interactions.", title="[bold green]🎉 Order Fulfillment Complete![/bold green]", border_style="green"))
 returngrid

# --- Main Execution ---
warehouse_layout = [
 "#######",
 "# D #",
 "# ### #",
 "#A#C# #",
 "# # #B#",
 "# P #",
 "#######",
]
order_to_fulfill = ['A', 'B']
final_grid = fulfill_order(warehouse_layout, order_to_fulfill, visualize_waves=True)

# --- Optional: LLM Interpretation ---
console.print("\n--- 🤖 LLM Interpretation的最终 State ---")
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1")
summary_prompt = ChatPromptTemplate.from_template("你is一个logistics manager. Briefly总结outcome的following order fulfillment report.\n\nOrder: {order}\n最终 Warehouse State: All itemsfromorder have been moved来packing station. Items AandB were retrieved. Original locations were {loc_A}and{loc_B}. floor is now clear.")
summary_chain = summary_prompt | llm
final_summary = summary_chain.invoke({
 "order": order_to_fulfill, 
 "loc_A": WarehouseGrid(warehouse_layout).item_locations['A'],
 "loc_B": WarehouseGrid(warehouse_layout).item_locations['B']
}).content
console.print(Markdown(final_summary))

 Path Wave Propagation (Stabilized在Tick #17) 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┃ █ ┃ █ ┃ █ ┃ █ ┃ █ ┃ █ ┃ █ ┃
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┠─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────┨
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┃ 7 ┃ 6 ┃ 5 ┃ D ┃ 5 ┃ 6 ┃ █ ┃
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┠─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────┨
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┃ 6 ┃ 5 ┃ 4 ┃ 3 ┃ 4 ┃ 5 ┃ 6 ┃
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┠─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────┨
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┃ A ┃ 4 ┃ 3 ┃ 2 ┃ C ┃ 6 ┃ 7 ┃
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┠─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────┨
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┃ 6 ┃ 5 ┃ 4 ┃ 1 ┃ 5 ┃ B ┃ 8 ┃
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┠─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────┨
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┃ 7 ┃ 6 ┃ 5 ┃ P ┃ 6 ┃ 7 ┃ 8 ┃
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┠─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────┨
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┃ █ ┃ █ ┃ █ ┃ █ ┃ █ ┃ █ ┃ █ ┃
┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

 步骤 1: Fulfill Item 'A' 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🌊 Computing path wave从Packing Station... ┃
┃ 🚚 Found path为item A. Moving along gradient... ┃
┃ Path: (3, 0) -> (3, 1) -> (3, 2) -> (4, 2) -> (5, 2) -> (5, 3) ┃
┃ ✅ Item 'A'有been moved来packing station. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 步骤 2: Fulfill Item 'B' 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🌊 Computing path wave从Packing Station... ┃
┃ 🚚 Found path为item B. Moving along gradient... ┃
┃ Path: (4, 5) -> (4, 4) -> (4, 3) -> (5, 3) ┃
┃ ✅ Item 'B'有been moved来packing station. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

 🎉 Order Fulfillment Complete! 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 系统 成功 fulfilled order为items ['A', 'B'] ┃
┃通过emergently computing paths through local cell interactions. ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


--- 🤖 LLM Interpretation的最终 State ---


order 为items 一个和B 有被successfully fulfilled. Item 一个was retrieved 从its shelf 在coordinates (3, 0) 和transported along 一个6-step path 到packing station. Subsequently, item B 被重新trieved 从(4, 5) 和moved along 一个4-step path 到same destination. warehouse floor 是now clear 和ready 为the next order.

### 结果分析

这detailed implementation perfectly showcases unique nature的元胞自动机为problem-solving:

1. **No Central Planner:** 在no point 做我们use 一个global pathfinding algorithm like A*. 我们never calculated 一个path在一个top-down manner. optimal path 是一个*emergent property*的grid itself.

2. **信息 作为一个Wave:** `propagate_path_wave`函数is key. visualization shows how 'distance'从packing station spreads across grid tick通过tick, flowing around obstacles naturally. 这是 "computational fabric"在work. grid 有essentially computed shortest path从*every single empty square*来packing station simultaneously.

3. **Simple Agent, Complex Behavior:** "carrier" 那个moves item 是incredibly 简单的. Its only logic 是"find neighbor与lowest number和move there." All 复杂的 environmental reasoning 是already encoded into grid's state通过path wave.

4. **Adaptability:** 如果 我们were来change warehouse layout通过adding 一个new obstacle，我们wouldn't need来rewrite 一个complex pathfinding algorithm. 我们would simply re-run wave propagation,和path values 会automatically和correctly flow around新的obstacle, demonstrating 系统's inherent adaptability.

这是 一个fundamental shift从traditional 代理 design. Instead的building one smart 代理 那个navigates 一个dumb environment，我们构建一个smart environment composed的many dumb 代理那个collectively solves problem.

## 总结

在这个notebook中，我们have built 一个fully-realized **元胞自动机 / Grid-Based Agent System**. 我们moved beyond theory和implemented 一个practical solution一个complex spatial reasoning problem, warehouse logistics.

我们已经 seen firsthand how 复杂的, goal-oriented behavior 可以emerge从synchronous execution的简单的, local rules across 一个grid的mini-agents. concepts的**wave propagation**和**gradient descent** 是not explicitly programmed在一个top-down manner但were natural result的cellular automata's evolution.

这种架构, while不suited为all problems, 是exceptionally 强大的为任务 involving spatial reasoning, simulation,和optimization在dynamic environments. It encourages us来think的代理ic 系统less 作为individual "bots"和more 作为一个**programmable, computational environment** 那个can 是configured来solve problems在一个massively parallel和adaptive way.